In [1]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import chromedriver_binary
import pandas as pd
import pprint
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
years = ['2005']
years.reverse()
berlin_url = 'https://www.bmw-berlin-marathon.com/en/impressions/statistics-and-history/results-archive/'
driver = webdriver.Chrome()
driver.get(berlin_url)
#accept cookies
time.sleep(1)
driver.find_element_by_xpath('//button[@data-cookiefirst-action="accept"]').click()
time.sleep(1)
for year in years:
    #get specific year
    driver.find_element_by_xpath('//select[@class="events border-input"]').click()
    time.sleep(1)
    driver.find_element_by_xpath(f"//option[contains(text(),{year})]").click()
    time.sleep(2)
    master_results = []
    #get number of pages
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pages = soup.find_all('a',{"class": "page-link"})
    pages_text = [res.getText(',') for res in pages]
    #open all the splits
    for i in range(int(pages_text[7])-2):
        time.sleep(1)
        html = driver.find_element_by_tag_name('html')
        html.send_keys(Keys.PAGE_UP)
        time.sleep(1)
        btns = driver.find_elements(By.XPATH, f'//td[@class= " details-control"]')
        for button in btns:
            button.click()
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        #scrape current page
        results = soup.find_all('tr')
        results_text = [res.getText(',') for res in results]
        accum = 0
        for i in range(21):
            master_results.append([results_text[accum] + ',' + results_text[accum+1]])
            accum += 2

        time.sleep(.2)

        #next page
        
        next = driver.find_element_by_xpath("//*[contains(text(), 'Next')]")
        actions = ActionChains(driver)
        actions.move_to_element(next).perform()
        next.click()
    #format data
    master_results_flat = [x for x in master_results]
    for i in range(len(master_results_flat)):
        master_results_flat[i] = master_results_flat[i][0].split(',')

    for i in range(20):
        for i in master_results_flat:
            if len(i) == 12:
                master_results_flat.remove(i)
            elif len(i) == 28:
                del i[5]
            elif len(i) != 27:
                master_results_flat.remove(i)

    #save to dataframe and csv
    df = pd.DataFrame(master_results_flat, columns = ['Overall_Place', 'First_Name','Last_Name', 'Country','SN', 'Sex','Net_time','Gross_time','split_times', '5k', '5k_split','10k', '10k_split','15k', '15k_split','20k', '20k_split','HM', 'HM_split','25k', '25k_split','30k', '30k_split','35k', '35k_split','40k', '40k_split'])
    
    df.to_csv(f'data/berlin_scrape_results/berlin_{year}.csv', header=True, index=False)
#print(master_results)


time.sleep(5)
driver.close()
driver.quit()

UnexpectedAlertPresentException: Alert Text: DataTables warning: table id=DataTables_Table_1 - Ajax error. For more information about this error, please see http://datatables.net/tn/7
Message: unexpected alert open: {Alert text : DataTables warning: table id=DataTables_Table_1 - Ajax error. For more information about this error, please see http://datatables.net/tn/7}
  (Session info: chrome=101.0.4951.64)


In [7]:
master_results_flat = [x for x in master_results]
for i in range(len(master_results_flat)):
    master_results_flat[i] = master_results_flat[i][0].split(',')

for i in range(20):
    for i in master_results_flat:
        if len(i) == 12:
            master_results_flat.remove(i)
        elif len(i) == 28:
            del i[5]
        elif len(i) != 27:
            master_results_flat.remove(i)

#save to dataframe and csv
df = pd.DataFrame(master_results_flat, columns = ['Overall_Place', 'First_Name','Last_Name', 'Country','SN', 'Sex','Net_time','Gross_time','split_times', '5k', '5k_split','10k', '10k_split','15k', '15k_split','20k', '20k_split','HM', 'HM_split','25k', '25k_split','30k', '30k_split','35k', '35k_split','40k', '40k_split'])
    
df.to_csv(f'berlin_2005_to_page_521.csv', header=True, index=False)